In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import numpy as np

## Loading data and labels

In [5]:
#https://keras.io/api/datasets/cifar100/
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode="fine")

In [6]:
label = [
    "apple",
    "aquarium_fish",
    "baby",
    "bear",
    "beaver",
    "bed",
    "bee",
    "beetle",
    "bicycle",
    "bottle",
    "bowl",
    "boy",
    "bridge",
    "bus",
    "butterfly",
    "camel",
    "can",
    "castle",
    "caterpillar",
    "cattle",
    "chair",
    "chimpanzee",
    "clock",
    "cloud",
    "cockroach",
    "couch",
    "crab",
    "crocodile",
    "cup",
    "dinosaur",
    "dolphin",
    "elephant",
    "flatfish",
    "forest",
    "fox",
    "girl",
    "hamster",
    "house",
    "kangaroo",
    "keyboard",
    "lamp",
    "lawn_mower",
    "leopard",
    "lion",
    "lizard",
    "lobster",
    "man",
    "maple_tree",
    "motorcycle",
    "mountain",
    "mouse",
    "mushroom",
    "oak_tree",
    "orange",
    "orchid",
    "otter",
    "palm_tree",
    "pear",
    "pickup_truck",
    "pine_tree",
    "plain",
    "plate",
    "poppy",
    "porcupine",
    "possum",
    "rabbit",
    "raccoon",
    "ray",
    "road",
    "rocket",
    "rose",
    "sea",
    "seal",
    "shark",
    "shrew",
    "skunk",
    "skyscraper",
    "snail",
    "snake",
    "spider",
    "squirrel",
    "streetcar",
    "sunflower",
    "sweet_pepper",
    "table",
    "tank",
    "telephone",
    "television",
    "tiger",
    "tractor",
    "train",
    "trout",
    "tulip",
    "turtle",
    "wardrobe",
    "whale",
    "willow_tree",
    "wolf",
    "woman",
    "worm"
]

In [8]:
i = 1500
label[y_train[i][0]]

'leopard'

## Data Arrangment

In [9]:
data = np.append(x_train.reshape((50000,-1)), y_train, axis=1)
train, val = train_test_split(data, test_size = 0.2,random_state=43)

test = np.append(x_test.reshape((10000,-1)), y_test, axis=1)

In [10]:
print(train.shape)
print(val.shape)
print(test.shape)

(40000, 3073)
(10000, 3073)
(10000, 3073)


In [11]:
X_train = train[:,:-1]
Y_train = train[:, -1]
X_val = val[:,:-1]
Y_val = val[:, -1]
X_test = test[:,:-1]
Y_test = test[:, -1]

print(X_train.shape, Y_train.shape)

(40000, 3072) (40000,)


# Logisitc Regression

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


clf = LogisticRegression(random_state=0)
clf = clf.fit(X_train, Y_train)

y_pred = clf.predict(X_val)
print("validation accuarcy:",accuracy_score(Y_val, y_pred))

validation accuarcy: 0.1716


C:\Users\saifa\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Conclusion
Really bad accuracy of 17% with Logistic Regression.

# Baseline Model

In [39]:
inputShape = [X_train[i,:].shape[0]]
print(inputShape)
classes_num = 100

model = Sequential()

model.add(layers.InputLayer(input_shape = inputShape)) # add first the input shape
model.add(layers.Dense(3000, activation="relu", kernel_initializer=tf.keras.initializers.HeNormal(), name="layer1"))
model.add(layers.Dense(2500, activation="relu", kernel_initializer=tf.keras.initializers.HeNormal(), name="layer2"))
model.add(layers.Dense(2000, activation="relu", kernel_initializer=tf.keras.initializers.HeNormal(), name="layer3"))
model.add(layers.Dense(1000, activation="relu", kernel_initializer=tf.keras.initializers.HeNormal(), name="layer4"))
model.add(layers.Dense(500, activation="relu", kernel_initializer=tf.keras.initializers.HeNormal(), name="layer5"))
model.add(layers.Dense(classes_num, activation="softmax" , kernel_initializer=tf.keras.initializers.HeNormal(), name="output_layer"))

model.summary()

[3072]
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3000)              9219000   
                                                                 
 layer2 (Dense)              (None, 2500)              7502500   
                                                                 
 layer3 (Dense)              (None, 2000)              5002000   
                                                                 
 layer4 (Dense)              (None, 1000)              2001000   
                                                                 
 layer5 (Dense)              (None, 500)               500500    
                                                                 
 output_layer (Dense)        (None, 100)               50100     
                                                                 
Total params: 24,275,100
Trainable params: 24,2

In [42]:
model.compile( optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [43]:
results = model.evaluate(X_test, Y_test, batch_size=128)

79/79 [==============================] - 2s 28ms/step - loss: 403.4444 - sparse_categorical_accuracy: 0.0099


Given that its completely random, it makes sense that the accuracy is pretty much 1% before any training as there are 100 classes to choose from and its a probability of 1 in a 100 to choose a class. 

In [44]:
history = model.fit(
    X_train,
    Y_train,
    epochs=50,
    batch_size=128,
    validation_data=(X_val, Y_val),
)

Epoch 1/50
313/313 [==============================] - 38s 119ms/step - loss: 30.6676 - sparse_categorical_accuracy: 0.0317 - val_loss: 7.3626 - val_sparse_categorical_accuracy: 0.0456
Epoch 2/50
313/313 [==============================] - 37s 120ms/step - loss: 6.0432 - sparse_categorical_accuracy: 0.0554 - val_loss: 5.4345 - val_sparse_categorical_accuracy: 0.0543
Epoch 3/50
313/313 [==============================] - 39s 125ms/step - loss: 4.8162 - sparse_categorical_accuracy: 0.0811 - val_loss: 4.9235 - val_sparse_categorical_accuracy: 0.0719
Epoch 4/50
313/313 [==============================] - 39s 123ms/step - loss: 4.3111 - sparse_categorical_accuracy: 0.1075 - val_loss: 4.4057 - val_sparse_categorical_accuracy: 0.0984
Epoch 5/50
313/313 [==============================] - 38s 121ms/step - loss: 4.0221 - sparse_categorical_accuracy: 0.1350 - val_loss: 4.4139 - val_sparse_categorical_accuracy: 0.0958
Epoch 6/50
313/313 [==============================] - 37s 119ms/step - loss: 3.8348 

### Conclusion
As you can see my model's accuracy has reached 90% on training and 22% on validation, that's definitely an improvement over 1% with logisitic regression however it is really bad still. There seems to be a lot overfitting happening too as the model was trained well on the trianing data however didnt predict much of the validation. Next step is finding out how we can optimize our model for better results

## Optimized Model 

In [109]:
inputShape=(32,32,3)
print("our input shape",inputShape)
#here we can see that CNN take the input as 3D tensors not a flatten 1D tensor

model = Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=inputShape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(100, activation='softmax'))

# Checking the model summary
model.summary()

our input shape (32, 32, 3)
Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_11 (MaxPoo

In [110]:
X_train = train[:,:-1].reshape(-1, 32, 32, 3)
X_train.shape
X_val = val[:,:-1].reshape(-1, 32, 32, 3)
# Y_val = val[:, -1].reshape(-1, 32, 32, 3)
X_test = test[:,:-1].reshape(-1, 32, 32, 3)
# Y_test = test[:, -1].reshape(-1, 32, 32, 3)

In [111]:
cnn_model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [113]:
print("Fit model on training data")
history = cnn_model.fit(
    X_train, Y_train,
    batch_size=128,
    epochs=20,
    validation_data=(X_val, Y_val),
)

Fit model on training data
Epoch 1/20
313/313 [==============================] - 138s 439ms/step - loss: 1.5636 - sparse_categorical_accuracy: 0.5555 - val_loss: 1.5149 - val_sparse_categorical_accuracy: 0.5711
Epoch 2/20
313/313 [==============================] - 136s 434ms/step - loss: 1.5508 - sparse_categorical_accuracy: 0.5610 - val_loss: 1.5339 - val_sparse_categorical_accuracy: 0.5704
Epoch 3/20
313/313 [==============================] - 132s 420ms/step - loss: 1.5506 - sparse_categorical_accuracy: 0.5561 - val_loss: 1.5661 - val_sparse_categorical_accuracy: 0.5639
Epoch 4/20
313/313 [==============================] - 132s 423ms/step - loss: 1.5361 - sparse_categorical_accuracy: 0.5646 - val_loss: 1.5097 - val_sparse_categorical_accuracy: 0.5761
Epoch 5/20
313/313 [==============================] - 130s 416ms/step - loss: 1.5288 - sparse_categorical_accuracy: 0.5652 - val_loss: 1.5869 - val_sparse_categorical_accuracy: 0.5608
Epoch 6/20
313/313 [==============================] -

### Conclusion
My model is a little better with 59% accuracy having used a better model I found here 

https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-build-a-convnet-for-cifar-10-and-cifar-100-classification-with-keras.md 

that seems to be implementing a CNN however 59% is still not great but we're getting somewhere. I just need to understand these additional layers more as right now Im really confused about what these layers mean.

## Transfer Learning

Loading in an already trained model and setting `trainable=False` so to 'freeze' everything except the last layer which we will add next

In [116]:
import tensorflow_hub as hub
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", output_shape=[1280],
                   trainable=False)
])
model.build([None, 224, 224, 3])

Adding the last layer to help with the output we need

In [117]:
model.add(layers.Dense(classes_num, activation='softmax'))
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_18 (KerasLayer)  (None, 1280)             2257984   
                                                                 
 dense_20 (Dense)            (None, 100)               128100    
                                                                 
Total params: 2,386,084
Trainable params: 128,100
Non-trainable params: 2,257,984
_________________________________________________________________


In [118]:
model.compile( optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
model.fit(
    X_train,
    Y_train,
    epochs=5
)

# This does not work
#### I understood how transfer learning works in theory however I cant get it to work here as the model is trained on 224x224 pixel images while my images are 32x32 so I wasn't really sure what to do at this point, I thought about resizing but it'll cause my images to lose a ton of quality. I deduced that Ill have to find a model which is already trained on 32x32 images but I couldnt find it/didnt know where to look.

# References
https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4

https://keras.io/guides/transfer_learning/

https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-build-a-convnet-for-cifar-10-and-cifar-100-classification-with-keras.md

https://www.youtube.com/watch?v=LsdxvjLWkIY&ab_channel=codebasics